## HMM predictor training and usage
This notebook demonstrates the training and usage of a simple Hidden Markov Model predictor 

#### Import library

In [2]:
import pandas as pd
import numpy as np
from hmmlearn import hmm
from sklearn.utils import check_random_state

#### Setting parameters

In [3]:
max_iter = 100
n_components = 3
n_sample = 1000

#### Read data

In [4]:
df = pd.read_csv('data/weather-data.label.1541249364.csv')
vals = df['category'].values
n_obs_states = len(set(list(vals)))

#### Train model

In [5]:
model = hmm.GaussianHMM(n_components=n_components, covariance_type="full", n_iter=max_iter)
vals = vals.reshape(-1, 1)
model.fit(vals);

#### Extract paramaters from model

In [6]:
states = model.decode(vals)[1]
emmission_mat = np.zeros([n_components, n_obs_states])

vals = [_[0] for _ in vals]
for _x, _y in zip(states, vals):
    emmission_mat[_x][_y] += 1
# end for

for row in range(len(emmission_mat)):
    emmission_mat[row] = emmission_mat[row, :] / sum(emmission_mat[row, :])\
# end for

#### Make predictions

In [7]:
# sample from models
transmat_cdf = np.cumsum(model.transmat_, axis=1)
random_state = check_random_state(model.random_state)

next_state_prob = []
for i in range(n_sample):
    next_state = (transmat_cdf[states[-1]] > random_state.rand()).argmax()
    next_state_prob.append(next_state)
# end for

In [8]:
# compute probabilities
next_state_prob = {_state: next_state_prob.count(_state) for _state in set(next_state_prob)}

In [9]:
# cast to array format
swap = np.zeros([1, n_components])
for key, val in next_state_prob.items():
    swap[0, key] = val / n_sample
# end for
next_state_prob = swap

In [10]:
# total probability
prob = np.array(np.matrix(next_state_prob) * np.matrix(emmission_mat))[0]

In [11]:
prob

array([0.638     , 0.205     , 0.12134661, 0.03565339])